<a href="https://colab.research.google.com/github/jscuteri/big-data-challenge/blob/main/big_data_challenge_musical_instruments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# import the SQL

!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-06 21:37:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.73MB/s    in 0.2s    

2022-03-06 21:37:54 (5.73 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Create a new session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [4]:
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
instruments_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True)
instruments_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
#Remove duplicates and na values
instruments_df = instruments_df.dropna()
instruments_df = instruments_df.dropDuplicates()
instruments_df.printSchema()
instruments_df.show()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_ra

In [6]:
# Count the number of records (rows) in the dataset
instruments_df.count()

904663

In [7]:
# Transform the dataset to fit the tables in the schema file
from pyspark.sql.functions import to_date

#Reviews
review_id_table = instruments_df.select(["review_id",  "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10054OAXKUYEW|    7728342|B010SENIZG|     445952714| 2015-08-19|
|R1029CP64GCQVU|   30508010|B0007WOX22|     453662589| 2015-04-28|
|R102H3GGAZCZCF|   14087707|B000LUVV5W|     826742236| 2007-10-11|
| R102SE89P3L1Y|   36446966|B007GI37CC|     961614944| 2015-04-08|
|R1037LO2K5NA94|   48245990|B00OUDSD84|     832356666| 2015-04-20|
|R103XZMKT84REI|    1603388|B009FVRDGO|     838447674| 2015-07-12|
|R105WD1IDQJPBN|   42694168|B00BLQTZDA|      90998007| 2013-06-29|
|R106H5VHYJ4BR1|   35587657|B002B5609C|     874657578| 2012-12-19|
|R1085A4HD07EXZ|   17100472|B005OYOBMK|     380200852| 2015-04-20|
|R108BDPC3QH8S2|   41391110|B0002F4ML6|     331336631| 2011-01-11|
|R108GLO55MCWUT|   42200534|B003KB2RS4|     741116158| 2012-06-29|
|R109A70FDX4GIC|   27593357|B003VWJ2K8|     195373332| 2014-03

In [8]:
# Column types match (integer)
review_id_table = review_id_table.withColumn("customer_id", review_id_table["customer_id"].cast(IntegerType()))
review_id_table = review_id_table.withColumn("product_parent", review_id_table["product_parent"].cast(IntegerType()))

In [9]:
# Check types
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
#Products
from pyspark import SparkFiles
products = instruments_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003WZXSWM|Grover 205BC Mini...|
|B00ASHQQKI|SUPERNIGHT 5050 1...|
|B000T4BC1I|Aulos A211A/A511B...|
|B005I0HTCY|Neewer Black 3.5m...|
|B00JJ18VIE|Reliable Hardware...|
|B00US77MXI|Pick Geek Guitar ...|
|B00S5O7HZO|Glory B Flat Clar...|
|B007CPU1A0|Meinl Headliner T...|
|B0002E3D9Y|PACK A STAND Clar...|
|B00J3W1F96|Zildjian 5A Acorn...|
|B0006NMUJU|Shure Foam Windsc...|
|B0018SJLFS|Tung-Sol 12AX7 Pr...|
|B00N0HXT94|Akai Professional...|
|B0006GRRQI|Gator G-PG ACOUST...|
|B005DJOI8I|Neewer 3.5mm Hand...|
|B000WS1QC6|Yamaha PA130 120 ...|
|B00IEQ4CVK|4/4 Size Electric...|
|B0080GZPRY|VocoPro CARRY-OKE...|
|B00I4ZMW2C|San Fransisco Gia...|
|B000F9NBZC|Tapco by Mackie M...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# Check types
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [12]:
#Customers
customers = instruments_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   39007869|             1|
|   24827391|             4|
|   35913081|             1|
|   43381900|            18|
|    2601964|             8|
|   51065955|             1|
|   11599427|            23|
|   38267705|             4|
|   11909158|             5|
|   44681263|             4|
|   15206433|             2|
|   10095624|             7|
|   44216833|             1|
|   31783146|             3|
|   50335921|             5|
|    9938121|             5|
|   18304630|            19|
|   49287327|             1|
|    3342990|             1|
|   20613732|             1|
+-----------+--------------+
only showing top 20 rows



In [13]:
# Column types match (integer)
customers = customers.withColumn("customer_id", customers["customer_id"].cast(IntegerType()))
customers = customers.withColumn("customer_count", customers["customer_count"].cast(IntegerType()))

In [14]:
# Check types
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [15]:
#Vine

vine_table = instruments_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10054OAXKUYEW|          5|            0|          1|   N|
|R1029CP64GCQVU|          5|            0|          0|   N|
|R102H3GGAZCZCF|          5|            5|         10|   N|
| R102SE89P3L1Y|          5|            1|          1|   N|
|R1037LO2K5NA94|          4|            0|          0|   N|
|R103XZMKT84REI|          5|            0|          0|   N|
|R105WD1IDQJPBN|          5|            7|          7|   N|
|R106H5VHYJ4BR1|          4|            0|          0|   N|
|R1085A4HD07EXZ|          4|            4|          4|   N|
|R108BDPC3QH8S2|          5|            1|          2|   N|
|R108GLO55MCWUT|          5|            4|          4|   N|
|R109A70FDX4GIC|          5|            0|          0|   N|
|R109XDH629T5CC|          5|            0|          0|   N|
|R10B7VE7FC2NJE|          4|            

In [16]:
# Column types match (integer)
vine_table = vine_table.withColumn("star_rating", vine_table["star_rating"].cast(IntegerType()))
vine_table = vine_table.withColumn("helpful_votes", vine_table["helpful_votes"].cast(IntegerType()))
vine_table = vine_table.withColumn("total_votes", vine_table["total_votes"].cast(IntegerType()))

In [17]:
# Check types
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cbguk05yuhpc.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "Scoots37*", 
          "driver":"org.postgresql.Driver"}

In [19]:
# Write DataFrame to review_id table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
# Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [21]:
# Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [22]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)